Backlogg
===

- todo test format of date


In [12]:
import csv
import glob
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay # BDay for business day

#todo temporary definition of constants
def create_stock_df(nr_days_to_trade=100, start_date_str='2017-01-17', nr_days_of_history=10):
    """
    Wrapper for importing the stock data
    """

    return(import_stock_data(nr_days_of_history, nr_days_to_trade, start_date_str))


def calculateDates(nr_days_of_history, nr_days_to_trade, start_date_str):
    """
    Calculates correct start and end dates for the data frame.
    - The start date is moved backward in order to include the history_window for the first day of trading.
    """

    # create a start date
    start_date = pd.Timestamp(start_date_str)

    # add end date
    end_date = start_date + BDay(nr_days_to_trade)

    # include a history window for first day
    start_date = start_date - BDay(nr_days_of_history)

    return {'start_date':start_date,'end_date':end_date}


# todo do proper check for file not found
def import_stock_data(nr_days_of_history, nr_days_to_trade, start_date_str):
    """
    Imports the relevant data from the text file system as a pandas data frame
    """
    # OBS!!!!!! temporary limitation for the size of the data frame
    size_of_subset = 100
    
    # pick out the desired start and end dates
    dates = calculateDates(nr_days_of_history = nr_days_of_history, nr_days_to_trade = nr_days_to_trade, start_date_str = start_date_str)

    # list the files to be read
    path = "./data/full_history/*.csv"
    all_files = np.array(glob.glob(path))

    # create an empty list to add each (relevant) stock's data frame
    li=[]

    # go through all the files in the folder
    for filename in all_files[:size_of_subset]:
        # read in the file
        df = pd.read_csv(filename, parse_dates=[0])
        
        # basic trimming: drop all extra columns,rename the stock value column after the stock
        df = df.drop(labels=list(['volume','close','high','low','adjclose']), axis='columns') 
        stock_name = filename.split(sep='/')[3].split(sep='.')[0].lower()
        df = df.rename(columns={'open': stock_name})
        
        # fit to time window: selct only rows with relevant dates and ignore stocks without relevant values
        df = df.loc[(df['date'] > dates['start_date']) & (df['date'] < dates['end_date'])]
        if df.empty:
            continue
        
        
        # prepare for concatenation: prep to join on dates and add to list of dfs
        df=df.set_index('date')
        li.append(df)

    
    # combine into a single data frame
    df = pd.concat(li, join='outer', axis=1)

    # remove dates (anonymises the time window somewhat)
    #df.reset_index(drop=True)

    return(df)

#window = sys.argv[1]
#start_date = sys.argv[2]
create_stock_df()

,isd,men,dfnl,hud,intg,fun,iusb,evf,chsco,bhvn,...,wynd,otex,eigr,ebtc,ent,mhn,wneb,pkg,east,arct
0,15.51,11.32,NaN,54.380001,29.40,63.500000,50.290001,6.71,28.660000,NaN,...,34.474041,30.985001,12.000,38.450001,6.49,13.45,9.45,85.440002,1.91,16.10
1,15.56,11.36,NaN,53.790001,29.40,63.790001,50.380001,6.63,29.799999,NaN,...,34.415348,31.264999,12.000,38.860001,6.58,13.54,9.50,86.370003,1.90,15.89
2,15.49,11.36,NaN,54.009998,28.00,64.519997,50.320000,6.65,29.320000,NaN,...,34.528217,31.434999,11.900,38.279999,6.43,13.56,9.40,85.370003,1.80,16.66
3,15.61,11.46,NaN,55.470001,29.40,64.760002,50.439999,6.70,29.160000,NaN,...,34.577877,31.295000,11.950,37.970001,6.24,13.54,9.45,86.900002,1.80,15.33
4,15.64,11.48,NaN,58.250000,29.35,63.730000,50.430000,6.72,29.430000,NaN,...,34.668171,31.139999,11.525,36.119999,6.20,13.56,9.30,87.699997,1.85,15.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,15.55,11.83,20.950001,NaN,NaN,70.760002,51.070000,6.77,29.200001,24.570000,...,44.948082,32.540001,6.950,30.190001,3.22,13.80,10.10,102.019997,1.95,8.12
101,15.62,11.85,20.770000,NaN,NaN,70.650002,51.099998,6.78,29.100000,24.290001,...,45.322800,32.419998,6.850,29.540001,3.15,13.84,10.00,102.669998,2.02,7.84
102,15.63,11.89,20.900000,NaN,NaN,70.559998,50.950001,6.78,29.000000,25.459999,...,45.905193,32.709999,6.700,30.299999,3.08,13.89,10.00,102.250000,2.15,7.84
103,15.61,11.94,21.010000,NaN,NaN,71.000000,51.110001,6.76,29.059999,25.090000,...,46.428894,32.810001,7.050,30.410000,3.04,13.85,10.10,103.959999,2.00,8.47


In [2]:
df

NameError: name 'df' is not defined